In [1]:
import os
import glob
import pandas as pd
import yfinance as yf
import numpy as np

c:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
#%pip install yfinance --upgrade --no-cache-dir --user

In [2]:
InfosToCopy = pd.read_csv('orders.csv', delimiter=";", header=0)
assetsList = sorted(InfosToCopy['Ativo'].unique())

for position in range(len(assetsList)):
    data = yf.download(assetsList[position]+".SA", start="2022-09-26")
    #data = yf.download("TSLA34.SA", start="2022-09-26")
    local = "output1/"+assetsList[position]+".csv"
    data.to_csv(local,index=True,header=True)
    #print(position, len(assetsList))    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
data = yf.download("TSLA34.SA", start="2022-09-26")
local = "output1/TSLA34.csv"
data.to_csv(local,index=True,header=True)

In [ ]:
dateCsv = pd.read_csv('output1/BRFS3.csv')
dateList = sorted(dateCsv['Date'].unique())
print(dateList)


In [ ]:
df=pd.DataFrame(dateList, columns=['Date'])
df1 = pd.read_csv('output1/BRFS3.csv')
df2 = df1[['Date', 'Close']].copy().rename(columns={'Close': 'BRFS3'})
inner_join=pd.merge(df,df2, on='Date', how='inner')
inner_join

In [6]:
path = "output1"
csv_files = glob.glob(os.path.join(path, "*.csv"))
FlagOnOff = 0

for files in csv_files:
    df = pd.read_csv(files, header='infer', index_col=0)
    df['Ticker'] = files.split("\\")[-1].split(".")[0]
    print(files.split("\\")[-1].split(".")[0])
    if FlagOnOff == 0:
        df.to_csv('output2/append.csv', mode='w', header=True)
        FlagOnOff = 1
    else:
        df.to_csv('output2/append.csv', mode='a', header=False)

AALR3
AGRO3
BBDC4
BOAS3
BRFS3
BRKM5
BRML3
CURY3
DIRR3
EZTC3
GGBR3
GOAU4
IFCM3
INTB3
JBSS3
JHSF3
KEPL3
KLBN4
LVTC3
MDIA3
MILS3
MLAS3
MOVI3
MRFG3
MYPK3
OIBR3
PORT3
RANI3
RECV3
ROMI3
SBFG3
SOJA3
SUZB3
TPIS3
TSLA34
TTEN3
UNIP6
WEGE3


In [ ]:
InfosToCopy2 = pd.read_csv('listaComRSI.csv', delimiter=";", header=None)
FlagOnOff = 0
for position in range(len(InfosToCopy2)):
    local = "output1/"+InfosToCopy2.iloc[position][0]+".csv"
    df5 = pd.read_csv(local, index_col=0)
    df5['Ticker'] = InfosToCopy2.iloc[position][0]
    if FlagOnOff == 0:
        df5.to_csv('output3/filter5.csv', mode='a', header=True)
        FlagOnOff = 1
    else:
        df5.to_csv('output3/filter5.csv', mode='a', header=False)

In [ ]:
InfosToCopy = pd.read_csv('lista2.csv', delimiter=";", header=None)
for position in range(len(InfosToCopy)):
    data = yf.download(InfosToCopy.iloc[position][0]+".SA", period="max")
    local = "output1/"+InfosToCopy.iloc[position][0]+".csv"
    data.to_csv(local,index=True,header=True)
    print(position, len(InfosToCopy))

In [ ]:
def build_RSI(data, column='Close', window=9, path="XXXX"):    

    # Establish gains and losses for each day
    data['Variation'] = data[column].diff()
    data = data[1:]
    data['Gain'] = np.where(data['Variation'] > 0, data['Variation'], 0)
    data['Loss'] = np.where(data['Variation'] < 0, data['Variation'], 0)

    # Calculate simple averages so we can initialize the classic averages
    simple_avg_gain = data['Gain'].rolling(window).mean()
    simple_avg_loss = data['Loss'].abs().rolling(window).mean()

    classic_avg_gain = simple_avg_gain.copy()
    classic_avg_loss = simple_avg_loss.copy()

    for i in range(window, len(classic_avg_gain)):
        classic_avg_gain[i] = (classic_avg_gain[i - 1] * (window - 1) + data['Gain'].iloc[i]) / window
        classic_avg_loss[i] = (classic_avg_loss[i - 1] * (window - 1) + data['Loss'].abs().iloc[i]) / window
    
    # Calculate the RSI
    data['Simple RS'] = simple_avg_gain / simple_avg_loss
    data['Classic RS'] = classic_avg_gain / classic_avg_loss

    data['Simple RSIf'] = 100 - (100 / (1 + data['Simple RS']))
    data['Classic RSIf'] = 100 - (100 / (1 + data['Classic RS']))

    data['Ticker'] = path.split("\\")[-1].split(".")[0]

    low_min = data['Low'].rolling(14).min()
    high_max = data['High'].rolling(14).max()
    
    # Fast Stochastic
    data['k_fast'] = 100 * (data['Close'] - low_min)/(high_max - low_min)

    # Slow Stochastic
    data['d_slow'] = data['k_fast'].rolling(3).mean()

    # Export to CSV
    local = "output2/"+path.split("\\")[-1]
    data.to_csv(local, index=True, sep=",", encoding='utf-8')

In [ ]:
path = "output1"
csv_files = glob.glob(os.path.join(path, "*.csv"))

for files in csv_files:
    df = pd.read_csv(files, header='infer', index_col=0)
    build_RSI(data=df, column="Close", window=9, path=files.split("\\")[-1])

In [ ]:
df = pd.read_csv("output2\BRFS3.csv")
#rslt_df = df[(df['Classic RSIf'] <= 15)] #164
#rslt_df = df[(df['Classic RSIf'] <= 15) & (df['k_slow'] <= 15)] #142
rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)] #145
df.tail(30)

In [ ]:
df = pd.read_csv("output2\ITUB4.csv")
rslt_df = df[(df['Classic RSIf'] <= 15)] #187
#rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)] #145
rslt_df.tail(30)

In [ ]:
path = "output2"
csv_files = glob.glob(os.path.join(path, "*.csv"))

for files in csv_files:
    df = pd.read_csv(files, header='infer', index_col=0)
    rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)]
    rslt_df.to_csv('output3/filter1c.csv', mode='a', header=True)
    rslt_df = df[(df['Classic RSIf'] >= 85) & (df['d_slow'] >= 85)]
    rslt_df.to_csv('output3/filter1v.csv', mode='a', header=True)
    #print(files)
    #build_RSI(data=df, column="Close", window=9, path=files.split("\\")[-1])

In [ ]:
df = pd.read_csv("output3/filter1c.csv")
df = df[(df['Date'] != "Date")]
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Adj Close'] = pd.to_numeric(df['Adj Close'])
df['Volume'] = pd.to_numeric(df['Volume'])
df['Variation'] = pd.to_numeric(df['Variation'])
df['Gain'] = pd.to_numeric(df['Gain'])
df['Loss'] = pd.to_numeric(df['Loss'])
df['Simple RS'] = pd.to_numeric(df['Simple RS'])
df['Classic RS'] = pd.to_numeric(df['Classic RS'])
df['Simple RSIf'] = pd.to_numeric(df['Simple RSIf'])
df['Classic RSIf'] = pd.to_numeric(df['Classic RSIf'])
df['k_fast'] = pd.to_numeric(df['k_fast'])
df['d_slow'] = pd.to_numeric(df['d_slow'])
df['Category'] = "C"
df = df[(df['Volume'] >= 1000000)]
df.to_csv('output3/filter2c.csv', header=True)

# Volume >= 1.000.000 - 1716 linhas
# Qualquer Volume - 58 mil linhas
# Volume >= 10.000 - 6006 linhas

In [ ]:
df = pd.read_csv("output3/filter1v.csv")
df = df[(df['Date'] != "Date")]
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Adj Close'] = pd.to_numeric(df['Adj Close'])
df['Volume'] = pd.to_numeric(df['Volume'])
df['Variation'] = pd.to_numeric(df['Variation'])
df['Gain'] = pd.to_numeric(df['Gain'])
df['Loss'] = pd.to_numeric(df['Loss'])
df['Simple RS'] = pd.to_numeric(df['Simple RS'])
df['Classic RS'] = pd.to_numeric(df['Classic RS'])
df['Simple RSIf'] = pd.to_numeric(df['Simple RSIf'])
df['Classic RSIf'] = pd.to_numeric(df['Classic RSIf'])
df['k_fast'] = pd.to_numeric(df['k_fast'])
df['d_slow'] = pd.to_numeric(df['d_slow'])
df['Category'] = "V"
df = df[(df['Volume'] >= 1000000)]
df.to_csv('output3/filter2v.csv', header=True)

# Volume >= 1.000.000 - 1716 linhas
# Qualquer Volume - 58 mil linhas
# Volume >= 10.000 - 6006 linhas

In [ ]:
df = pd.read_csv("output3/filter2v.csv", index_col=0)
df.head(30)

In [ ]:
dfv = pd.read_csv("output3/filter2v.csv", index_col=0)
dfc = pd.read_csv("output3/filter2c.csv", index_col=0)

dfc.to_csv('output3/filter3.csv', mode='a', header=True)
dfv.to_csv('output3/filter3.csv', mode='a', header=False)

df3 = pd.read_csv("output3/filter3.csv", index_col=0)
df3.tail(30)

In [ ]:
header = ['Ticker', 'Date', 'Category', 'Adj Close']
df3.to_csv('output3/filter4.csv', header=True, columns=header)

In [ ]:
df4 = pd.read_csv("output3/filter4.csv", index_col=0)
df4.head(30)

In [ ]:
InfosToCopy2 = pd.read_csv('listaComRSI.csv', delimiter=";", header=None)
FlagOnOff = 0
for position in range(len(InfosToCopy2)):
    local = "output1/"+InfosToCopy2.iloc[position][0]+".csv"
    df5 = pd.read_csv(local, index_col=0)
    df5['Ticker'] = InfosToCopy2.iloc[position][0]
    if FlagOnOff == 0:
        df5.to_csv('output3/filter5.csv', mode='a', header=True)
        FlagOnOff = 1
    else:
        df5.to_csv('output3/filter5.csv', mode='a', header=False)
    